In [1]:
%pip install requests azure-identity msgraph-sdk neo4j graphdatascience

Note: you may need to restart the kernel to use updated packages.


In [50]:
import asyncio
import configparser
from msgraph.generated.models.o_data_errors.o_data_error import ODataError

from configparser import SectionProxy
from azure.identity import ClientSecretCredential
from msgraph import GraphServiceClient
from msgraph.generated.users.item.user_item_request_builder import UserItemRequestBuilder
from msgraph.generated.users.users_request_builder import UsersRequestBuilder
from msgraph.generated.sites.sites_request_builder import SitesRequestBuilder
from kiota_abstractions.base_request_configuration import RequestConfiguration

In [68]:
class Graph:
    settings: SectionProxy
    client_credential: ClientSecretCredential
    app_client: GraphServiceClient

    def __init__(self, config: SectionProxy):
        self.settings = config
        client_id = self.settings['clientId']
        tenant_id = self.settings['tenantId']
        client_secret = self.settings['clientSecret']

        self.client_credential = ClientSecretCredential(tenant_id, client_id, client_secret)
        self.app_client = GraphServiceClient(self.client_credential)
    
    def get_app_only_token(self):
        graph_scope = 'https://graph.microsoft.com/.default'
        access_token = self.client_credential.get_token(graph_scope)
        return access_token.token

    def get_users(self):
        query_params = UsersRequestBuilder.UsersRequestBuilderGetQueryParameters(
            select=['displayName', 'id', 'mail'],
            top=25,
            orderby=['displayName']
        )
        request_config = UsersRequestBuilder.UsersRequestBuilderGetRequestConfiguration(
            query_parameters=query_params
        )

        users = self.app_client.users.get(request_configuration=request_config)
        return users


In [69]:
config = configparser.ConfigParser()
config.read(['config.cfg'])
azure_settings = config['azure']

graph: Graph = Graph(azure_settings)

In [70]:
try:
    token = graph.get_app_only_token()
    print(f'App-only token: {token}')
except ODataError as odata_error:
    print(odata_error.error.code, odata_error.error.message)

App-only token: eyJ0eXAiOiJKV1QiLCJub25jZSI6ImpBSUZKUUpVOUhNUFh3WnNQMkpGWURRdURIU1ktTVY2a19tSGxOcDVsenciLCJhbGciOiJSUzI1NiIsIng1dCI6IjNQYUs0RWZ5Qk5RdTNDdGpZc2EzWW1oUTVFMCIsImtpZCI6IjNQYUs0RWZ5Qk5RdTNDdGpZc2EzWW1oUTVFMCJ9.eyJhdWQiOiJodHRwczovL2dyYXBoLm1pY3Jvc29mdC5jb20iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC82MTZmMjdlMy1kZmQzLTQ2ZTYtOWU3Ny1lMmE1NThlNWQ2MmMvIiwiaWF0IjoxNzI5NTg5NDk4LCJuYmYiOjE3Mjk1ODk0OTgsImV4cCI6MTcyOTU5MzM5OCwiYWlvIjoiazJCZ1lCQnRPcWk2TDM1THlBZkhoS3RURjd4ZkJBQT0iLCJhcHBfZGlzcGxheW5hbWUiOiJweWNvbi1ub2Rlcy1kZW1vIiwiYXBwaWQiOiI1ZmU4YzE4OS02NmQzLTQ4ZjQtYTlkMC1hZTU4ZjAzZGM5MTYiLCJhcHBpZGFjciI6IjEiLCJpZHAiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC82MTZmMjdlMy1kZmQzLTQ2ZTYtOWU3Ny1lMmE1NThlNWQ2MmMvIiwiaWR0eXAiOiJhcHAiLCJvaWQiOiI5MGFkOWE3ZC03MjhkLTQwMWMtYTNiZS0xODVlOWVhYTYwNWEiLCJyaCI6IjAuQWNZQTR5ZHZZZFBmNWthZWQtS2xXT1hXTEFNQUFBQUFBQUFBd0FBQUFBQUFBQURHQUFBLiIsInJvbGVzIjpbIlVzZXIuUmVhZC5BbGwiXSwic3ViIjoiOTBhZDlhN2QtNzI4ZC00MDFjLWEzYmUtMTg1ZTllYWE2MDVhIiwidGVuYW50X3JlZ2lvbl9zY29wZSI6IkF

List users

In [66]:
try:
    users = await graph.get_users()
    for user in users:
        print(f'User: {user}')
except ODataError as odata_error:
    print(odata_error.error.code, odata_error.error.message)

Authorization_RequestDenied Insufficient privileges to complete the operation.


Get a site using the site ID

In [48]:
result = await graph.app_client.sites.by_site_id('d4782aca-aacc-4f4f-a6d1-af9e0f41c4be').get()

ODataError: 
        APIError
        Code: 403
        message: None
        error: MainError(additional_data={}, code='AccessDenied', details=None, inner_error=InnerError(additional_data={}, client_request_id='b9a00df4-45b4-4207-94fb-3ccf313e7f12', date=DateTime(2024, 10, 22, 9, 10, 39, tzinfo=Timezone('UTC')), odata_type=None, request_id='ca0193ac-9571-4ebf-b890-d0583128a08c'), message='Either scp or roles claim need to be present in the token.', target=None)
        